<a href="https://colab.research.google.com/github/libclick/DATA205/blob/main/Towing_in_Montgomery_County.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import json
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [110]:
# get data: Towing Report from Montgomery Data
towing = pd.DataFrame(requests.get('https://data.montgomerycountymd.gov/resource/i6vn-3s6e.json').json())
towing.head()

,tow_date,vehicle_year,vehicle_make,vehicle_model,tow_reason,story_company,notes,incident_date,location,city,geo_location,:@computed_region_vu5j_pcmz,:@computed_region_tx5f_5em3,:@computed_region_kbsp_ykn9,:@computed_region_d7bw_bq6x,:@computed_region_rbt8_3x7n
0,2020-12-31T00:00:00.000,2011,CHEV,SIL,NOP,Henry's,NOP // HENRYS 301-869-4800 // TCL // VIN# NOT ...,0020-12-31T06:21:00.000,2386 GLENMONT CIR,SILVER SPRING,"{'type': 'Point', 'coordinates': [-77.047999, ...",4,4,23,17,1
1,2020-12-31T00:00:00.000,2012,NISS,ALT,NOP,Speedy,SPEEDY/3015881322/TCL / NOP,0020-12-31T12:46:00.000,8115 FENTON ST,SILVER SPRING,"{'type': 'Point', 'coordinates': [-77.024011, ...",5,5,7,3065,1
2,2020-12-31T00:00:00.000,2008,HYUN,ACC,REPO,CODY,TCL 301-473-4322 // REPO // FIN: ROYAL TRUST 3...,0020-12-31T08:13:00.000,9309 WILLOW CREEK DR,MONTGOMERY VILLAGE,"{'type': 'Point', 'coordinates': [-77.194533, ...",2,2,38,28,1
3,2020-12-31T00:00:00.000,2015,TOYT,VNZ,REPO,TCL,REPO // ALLY FINANCIAL // 800-354-0759 PAR S...,0020-12-31T02:56:00.000,905 CLOPPER RD,GAITHERSBURG,"{'type': 'Point', 'coordinates': [-77.226827, ...",3,3,36,33,21
4,2020-12-31T00:00:00.000,2014,TOYT,COR,NOP,tcl,PRESITGE 3013779285 NOP TCL,0020-12-31T03:12:00.000,418 ALLIED PL,GAITHERSBURG,"{'type': 'Point', 'coordinates': [-77.191331, ...",3,3,36,35,21


In [111]:
#capitalize on first letter of each string in towing.notes, towing.story_company, towing.location, towing.city
#NOTE ISSUE WITH APOSTROPHE IN Henry'S - fix later with if statement for this one case?
towing.loc[:,"notes"] = towing.loc[:,"notes"].str.title()
towing.loc[:,"story_company"] = towing.loc[:,"story_company"].str.title()
towing.loc[:,"location"] = towing.loc[:,"location"].str.title()
towing.loc[:,"city"] = towing.loc[:,"city"].str.title()
towing.head()
towing.dtypes

tow_date                       object
vehicle_year                   object
vehicle_make                   object
vehicle_model                  object
tow_reason                     object
story_company                  object
notes                          object
incident_date                  object
location                       object
city                           object
geo_location                   object
:@computed_region_vu5j_pcmz    object
:@computed_region_tx5f_5em3    object
:@computed_region_kbsp_ykn9    object
:@computed_region_d7bw_bq6x    object
:@computed_region_rbt8_3x7n    object
dtype: object

In [112]:
#TOO COMPLICATED EXPANDING COLUMMNS OUT AND ITERATING ACROSS WITH SERIES OF ACTIVE COMPANIES - KEEP CHUNKS FOR REFERENCE ONLY
# place note columns into notes_series; then create notes_df, a panda of notes column strings expanded into many columns
# notes_series = pd.Series(towing["notes"])
# notes_df = notes_series.str.split(pat=" ", n=10, expand=True)   
# print(notes_df)
#towing

In [113]:
company = pd.DataFrame(requests.get('https://data.montgomerycountymd.gov/resource/dngn-wp3e.json').json())
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   registration_type                            1000 non-null   object
 1   corporation_name                             793 non-null    object
 2   trade_name                                   996 non-null    object
 3   business_address                             958 non-null    object
 4   city                                         976 non-null    object
 5   state                                        976 non-null    object
 6   zip                                          571 non-null    object
 7   business_tel_no                              968 non-null    object
 8   niase_certified_mechanical_employees_number  1000 non-null   object
 9   issue_date                                   1000 non-null   object
 10  expire_date  

In [114]:
#convert expire date and issue date to datetime object
company['expire_date']= pd.to_datetime(company['expire_date'])
company['issue_date']= pd.to_datetime(company['issue_date'])

In [115]:
#get actively licensed companies only and place in active_company dataframe - REFERENCE
#import datetime as dt
#dt_cutoff = dt.datetime(2020, 12, 31, 11, 59, 59)
#active_company = company[(company["expire_date"] > company["issue_date"]) & (company["expire_date"] > dt_cutoff)]
#active_company.info()
#type(active_company)

In [116]:
#active_company.head()

In [117]:
#clean column data as before to "title" type capitalization - REFERENCE
#active_company.loc[:,"corporation_name"] = active_company.loc[:,"corporation_name"].str.title()
#active_company.loc[:,"trade_name"] = active_company.loc[:,"trade_name"].str.title()
#active_company.loc[:,"business_address"] = active_company.loc[:,"business_address"].str.title()
#active_company.loc[:,"city"] = active_company.loc[:,"city"].str.title()
#active_company.head()

In [118]:
#Create active list of trade names for search/match of "notes" in towing dataset; create active_series as well - REFERENCE
#active_names = list(active_company["trade_name"])
##actives_names = list(active_names)
#print(active_names)
#print(active_names.index)
#len(active_names)

#active_series = pd.Series(active_names)
#print(active_series)

In [119]:
# load json file from dataMontgomery: 2010 CENSUS DATASET will be used to link track code to latitude/longitude info for demographics dataset
codes = pd.DataFrame(requests.get('https://data.montgomerycountymd.gov/resource/y2z8-tsry.json').json())
codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   the_geom    614 non-null    object
 1   aian        614 non-null    object
 2   aland10     614 non-null    object
 3   asian       614 non-null    object
 4   awater10    614 non-null    object
 5   bg2010      614 non-null    object
 6   black       614 non-null    object
 7   blkgrp      614 non-null    object
 8   county      614 non-null    object
 9   hl          614 non-null    object
 10  latitude    614 non-null    object
 11  longitude   614 non-null    object
 12  mult_race   614 non-null    object
 13  name        614 non-null    object
 14  nh_aian     614 non-null    object
 15  nh_asian    614 non-null    object
 16  nh_black    614 non-null    object
 17  nh_mult_ra  614 non-null    object
 18  nh_nhopi    614 non-null    object
 19  nh_other    614 non-null    object
 20  nh_white  

In [120]:
codes.head()

,the_geom,aian,aland10,asian,awater10,bg2010,black,blkgrp,county,hl,latitude,longitude,mult_race,name,nh_aian,nh_asian,nh_black,nh_mult_ra,nh_nhopi,nh_other,nh_white,nhl,nhopi,objectid,one_race,other,shape_area,shape_len,state,stusab,sumlev,total,tract,white
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",0.0,16879105.0,114.0,56208.0,240317013171,120.0,1.0,31.0,55.0,39.19038773,-077.0813591,47.0,Block Group 1,0.0,114.0,120.0,42.0,0.0,3.0,936.0,1215.0,0,417.0,1223,10.0,182276687.152,83475.9553498,24.0,MD,150.0,1270.0,701317.0,979.0
1,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",41.0,839495.0,494.0,0.0,240317039022,1055.0,2.0,31.0,1025.0,39.03575897,-077.0434957,205.0,Block Group 2,21.0,488.0,1022.0,98.0,13.0,8.0,1078.0,2728.0,13,421.0,3548,417.0,9035363.68988,12949.145971,24.0,MD,150.0,3753.0,703902.0,1528.0
2,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",7.0,372206.0,50.0,0.0,240317022003,80.0,3.0,31.0,131.0,39.0110817,-077.0114020,49.0,Block Group 3,0.0,48.0,79.0,28.0,0.0,7.0,744.0,906.0,0,4.0,988,54.0,4005977.99431,10602.1353642,24.0,MD,150.0,1037.0,7.022E+5,797.0
3,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",10.0,1448076.0,201.0,1882.0,240317013132,193.0,2.0,31.0,197.0,39.13514709,-077.0777340,46.0,Block Group 2,1.0,201.0,192.0,34.0,1.0,1.0,982.0,1412.0,1,11.0,1563,80.0,15605889.222,16420.4494053,24.0,MD,150.0,1609.0,701313.0,1078.0
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",19.0,315666.0,139.0,0.0,240317034042,365.0,2.0,31.0,1439.0,39.05306625,-077.0708093,122.0,Block Group 2,3.0,138.0,339.0,35.0,0.0,16.0,207.0,738.0,0,14.0,2055,758.0,3397475.10699,7948.66279768,24.0,MD,150.0,2177.0,703404.0,774.0


In [121]:
#MODIFY TO LOAD CSV DIRECTLY - NO JSON
# read demographics dataset from formatted csv; to be translated with codes dataframe and combined with towing dataset 
#demographics = pd.read_csv("")  

In [122]:
x=-1
rowcount = 998
while x <= rowcount :
  x=x+1
  if towing.iloc[x, 5] == "Tcl" :
    print("get company from notes column")

 
  else : 
    print(x)
#towing.head()
#Series.str.replace(pat, repl, n=- 1, case=None, flags=0, regex=None)
                                              #towing("story_company") = string.match(active.series 

0
1
2
get company from notes column
get company from notes column
5
get company from notes column
7
8
9
10
get company from notes column
get company from notes column
get company from notes column
14
15
get company from notes column
get company from notes column
18
get company from notes column
get company from notes column
21
get company from notes column
23
24
25
26
get company from notes column
get company from notes column
get company from notes column
get company from notes column
31
get company from notes column
33
34
35
36
get company from notes column
38
get company from notes column
get company from notes column
41
42
43
44
45
46
47
48
get company from notes column
get company from notes column
get company from notes column
get company from notes column
get company from notes column
54
55
get company from notes column
57
58
59
60
61
get company from notes column
63
64
65
get company from notes column
get company from notes column
68
get company from notes column
get company fr

In [123]:
towing.head()

,tow_date,vehicle_year,vehicle_make,vehicle_model,tow_reason,story_company,notes,incident_date,location,city,geo_location,:@computed_region_vu5j_pcmz,:@computed_region_tx5f_5em3,:@computed_region_kbsp_ykn9,:@computed_region_d7bw_bq6x,:@computed_region_rbt8_3x7n
0,2020-12-31T00:00:00.000,2011,CHEV,SIL,NOP,Henry'S,Nop // Henrys 301-869-4800 // Tcl // Vin# Not ...,0020-12-31T06:21:00.000,2386 Glenmont Cir,Silver Spring,"{'type': 'Point', 'coordinates': [-77.047999, ...",4,4,23,17,1
1,2020-12-31T00:00:00.000,2012,NISS,ALT,NOP,Speedy,Speedy/3015881322/Tcl / Nop,0020-12-31T12:46:00.000,8115 Fenton St,Silver Spring,"{'type': 'Point', 'coordinates': [-77.024011, ...",5,5,7,3065,1
2,2020-12-31T00:00:00.000,2008,HYUN,ACC,REPO,Cody,Tcl 301-473-4322 // Repo // Fin: Royal Trust 3...,0020-12-31T08:13:00.000,9309 Willow Creek Dr,Montgomery Village,"{'type': 'Point', 'coordinates': [-77.194533, ...",2,2,38,28,1
3,2020-12-31T00:00:00.000,2015,TOYT,VNZ,REPO,Tcl,Repo // Ally Financial // 800-354-0759 Par S...,0020-12-31T02:56:00.000,905 Clopper Rd,Gaithersburg,"{'type': 'Point', 'coordinates': [-77.226827, ...",3,3,36,33,21
4,2020-12-31T00:00:00.000,2014,TOYT,COR,NOP,Tcl,Presitge 3013779285 Nop Tcl,0020-12-31T03:12:00.000,418 Allied Pl,Gaithersburg,"{'type': 'Point', 'coordinates': [-77.191331, ...",3,3,36,35,21
